In [6]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 12991 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_2_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_2_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [10]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [11]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20, 30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=135149, Thu Apr 16 18:34:26 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.013098752017508e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.719186449259273e+08 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.57633498e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.24387045e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.556487735788035e+08 1.0e+00 2.35e+00  2e+00  2e+00 0:00.4
   21    399 5.981861064815599e+07 1.0e+00 2.13e+00  2e+00  2e+00 0:03.4
   43    817 3.235395013192868e+07 1.1e+00 2.05e+00  2e+00  2e+00 0:07.5
   83   1577 9.310691631049335e+06 1.2e+00 1.75e+00  2e+00  2e+00 0:12.6
  100   1900 -3.667113265214223e+07 1.2e+00 1.59e+00  2e+00  2e+00 0:14.7
  155   2945 -1.174912568639405e+08 1.2e+00 1.09e+00  1e+00  1e+00 0:21.8
  200   3800 -2.092369501516079e+08 1.3e+00 7.21e-01  7e-01  7e-01 0:27.6
  272   5168 -2.928642937694924e+08 1.3e+00 3.54e-01  3e-01  4e-01 0:36.6
  300   5700 -3.070033825479084e+08 1.3e+00 2.70e-01  3e-01  3e-01 0:40.2
  386   7334 -3.229218741415586e+08 1.3e+00 1.23e-01  1e-01  1e-01 0:51.3
  400   7600 -3.237316492760064e+08 1.3e+00 1.07e-01  1e-01  1e-01 0:53.1
  498   9462 -3.264967302722172e+08 1.3e+00 4.30e-02  4e-02  4e-02 1:06.2
  500   9500 -3.265005534348848e+08 1.3e+00 4.24e-02  4e-02  4e-02 1:06.5
  600  11400 -3.269115811087610e+08 1.3e+0

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.02915659e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Thu Apr 16 18:59:16 2020)
final/bestever f-value = 2.029157e+08 2.029157e+08
incumbent solution: [ 2.84977625 -2.56417023  1.03748804  3.65468824 -4.58440459  0.54465087
  4.99404253  1.56133357 ...]
std deviations: [2.43727553 2.43781825 2.43754185 2.43767064 2.43741299 2.43745934
 2.43751037 2.43767272 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=115231, Thu Apr 16 18:59:16 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.110861592380759e+08 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 1.931262610871667e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

    3     57 1.202268438616688e+08 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   30    570 3.697977327902445e+07 1.1e+00 2.03e+00  2e+00  2e+00 0:03.4
   65   1235 5.975901492060423e+06 1.2e+00 1.92e+00  2e+00  2e+00 0:07.5
  100   1900 -2.715359307609436e+07 1.2e+00 1.54e+00  2e+00  2e+00 0:11.5
  153   2907 -1.219385959900584e+08 1.3e+00 1.10e+00  1e+00  1e+00 0:17.6
  200   3800 -2.168005628499134e+08 1.3e+00 6.89e-01  7e-01  7e-01 0:23.1
  270   5130 -2.907409006741362e+08 1.3e+00 3.74e-01  4e-01  4e-01 0:31.2
  300   5700 -3.026189497853231e+08 1.3e+00 2.92e-01  3e-01  3e-01 0:34.6
  387   7353 -3.216256610517313e+08 1.3e+00 1.29e-01  1e-01  1e-01 0:44.7
  400   7600 -3.235724580453871e+08 1.3e+00 1.15e-01  1e-01  1e-01 0:46.2
  500   9500 -3.264644342456749e+08 1.3e+00 4.54e-02  4e-02  5e-02 0:57.8
  600  11400 -3.269286767488996e+08 1.4e+00 1.75e-02  2e-02  2e-02 1:09.6
  700  13300 -3.269998634615442e+08 1.4e+00 6.82e-03  6e-03  7e-03 1:21.3
  800  15200 -3.270117067571600e+08 1.4e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.02915659e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.43e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.43e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Fri Apr 17 01:46:37 2020)
final/bestever f-value = 2.029157e+08 2.029157e+08
incumbent solution: [-4.85206497 -1.86390017 -3.8893756   3.01121697  3.02937675  3.34576348
 -0.54090773 -0.54400999 ...]
std deviations: [2.43478145 2.43509471 2.43505455 2.4350808  2.43431578 2.43511516
 2.43503841 2.43469646 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=228374, Fri Apr 17 01:46:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.267028932954244e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.440457844113991e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.71843094e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.30833326e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.029677081889347e+08 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   32    608 3.886743848865335e+07 1.1e+00 1.96e+00  2e+00  2e+00 0:03.4
   70   1330 3.711944367775777e+07 1.1e+00 1.79e+00  2e+00  2e+00 0:07.5
  100   1900 3.565085789108346e+07 1.2e+00 1.73e+00  2e+00  2e+00 0:10.7
  157   2983 3.422318106954584e+07 1.3e+00 1.38e+00  1e+00  1e+00 0:16.8
  200   3800 1.977677970984511e+07 1.3e+00 1.19e+00  1e+00  1e+00 0:21.4
  274   5206 1.873377680873570e+07 1.3e+00 9.91e-01  1e+00  1e+00 0:29.4
  300   5700 1.524916844172165e+07 1.3e+00 8.92e-01  9e-01  9e-01 0:32.2
  394   7486 1.206206864407070e+07 1.3e+00 6.88e-01  7e-01  7e-01 0:42.3
  400   7600 1.143451036936174e+07 1.3e+00 6.71e-01  6e-01  7e-01 0:42.9
  500   9500 8.546928881079853e+06 1.4e+00 4.85e-01  5e-01  5e-01 0:53.6
  600  11400 6.056901072538570e+06 1.5e+00 3.29e-01  3e-01  3e-01 1:04.3
  700  13300 4.345385868707672e+06 1.5e+00 2.43e-01  2e-01  3e-01 1:15.0
  800  15200 3.053181452245429e+06 1.5e+00 2.14e-01

In [12]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F2_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F2(X_Values.iloc[i,:200])
    SVM_Fun [i] = F2(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F2(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([50672203.99846563]), count=array([1])) 0.5459893753246737
SVM
ModeResult(mode=array([1.42251931e+08]), count=array([1])) 33838857.67073601
ELN
ModeResult(mode=array([2865738.1884947]), count=array([1])) 453669.5442832603
